In [1]:
import pandas as pd
import numpy as np


In [2]:
'''
Data properties: 8 classes (1,2,3,4,5,6,7,8), 8 features
X_train shape: (10800, 8)
y_train shape: (10800,)
X_test shape:  (1200, 8)
'''

train_data = pd.read_csv('./data/trn1.csv')
test_data = pd.read_csv('./data/tst1.csv', header=None)

# Prepare train and test data
X_train = train_data.iloc[:, :-1].values
y_train = train_data.iloc[:, -1].values
X_test = test_data.values


C = len(np.unique(y_train)) # number of classes = 8
N = X_train.shape[0] # number of samples = 10800
d = X_train.shape[1] #number of features = 8
I_matrix = np.identity(d)


class_probs = np.array([np.mean(y_train == i) for i in range(1, C + 1)]) # (8,)
# print(class_probs.shape)

class_means = np.array([np.mean(X_train[y_train == i], axis=0) for i in range(1, C + 1)]) #(8, 8)
# print(class_means.shape)

class_cov = np.array([np.cov(X_train[y_train == i].T) for i in range(1, C + 1)]) #(8, 8, 8)
# print(class_cov.shape)

common_cov = np.sum([class_probs[i]*class_cov[i] for i in range (C)], axis=0) #(8,8)
# print(common_cov.shape)

diag_cov = common_cov * I_matrix #(8,8)
# print(diag_cov.shape)



In [3]:

def discriminant_func(x, class_idx):
    x_diff = x - class_means[class_idx - 1] 
    cov_inv = np.linalg.inv(class_cov[class_idx - 1])   
    return -0.5 * np.log(np.linalg.det(class_cov[class_idx - 1])) - \
           0.5 * np.sum(x_diff @ cov_inv * x_diff, axis=1) + np.log(class_probs[class_idx - 1])

determinants = np.array([np.linalg.det(cov_matrix) for cov_matrix in class_cov])

discriminant_functions = np.array([discriminant_func(X_train, i) for i in range(1, C + 1)]).T

predicted_classes = np.argmax(discriminant_functions, axis=1) + 1

accuracy = np.sum(predicted_classes==y_train, axis=0)/y_train.shape[0]
print(f"Accuracy - Normal = {accuracy}")



Accuracy - Normal = 0.8969444444444444


# Common covariance matrix

In [4]:
def common_discriminant_func(x, class_idx):
    x_diff = x - class_means[class_idx - 1]
    cov_inv = np.linalg.inv(common_cov)
    return -0.5 * np.sum(x_diff.dot(cov_inv) * x_diff, axis=1) + np.log(class_probs[class_idx - 1])

determinants_common = np.linalg.det(common_cov)

common_discriminant_functions = np.array([common_discriminant_func(X_train, i) for i in range(1, C + 1)]).T

predicted_classes = np.argmax(common_discriminant_functions, axis=1) + 1

accuracy_common = np.sum(predicted_classes==y_train, axis=0)/y_train.shape[0]
print(f"Accuracy - Common = {accuracy_common}")


Accuracy - Common = 0.8283333333333334


In [5]:
#Generate test result3_1

common_discriminant_functions = np.array([common_discriminant_func(X_test, i) for i in range(1, C + 1)]).T
test_prediction = np.argmax(common_discriminant_functions, axis=1) + 1

# Add the predicted class to the testing data
result_data1 = np.column_stack((test_data, test_prediction))

# Save results to result3_1.csv
result_df1 = pd.DataFrame(result_data1)
result_df1.to_csv('./results/result3_1.csv', index=False, header=False)

# Diagonal matrix

In [6]:
def diag_discriminant_func(x, class_idx):
    x_diff = x - class_means[class_idx - 1]
    cov_inv = np.linalg.inv(diag_cov)
    return -0.5 * np.sum(x_diff.dot(cov_inv) * x_diff, axis=1) + np.log(class_probs[class_idx - 1])

determinants_diag = np.linalg.det(diag_cov)

diag_discriminant_functions = np.array([diag_discriminant_func(X_train, i) for i in range(1, C + 1)]).T

predicted_classes = np.argmax(diag_discriminant_functions, axis=1) + 1

accuracy_diag = np.sum(predicted_classes==y_train, axis=0)/y_train.shape[0]
print(f"Accuracy - Diag = {accuracy_diag}")


Accuracy - Diag = 0.8280555555555555


In [7]:
#Generate test result3

diag_discriminant_functions = np.array([diag_discriminant_func(X_test, i) for i in range(1, C + 1)]).T
test_prediction = np.argmax(diag_discriminant_functions, axis=1) + 1

# Add the predicted class to the testing data
result_data2 = np.column_stack((test_data, test_prediction))

# Save results to result3_2.csv
result_df2 = pd.DataFrame(result_data2)
result_df2.to_csv('./results/result3_2.csv', index=False, header=False)